# 1. Read your exported json file from the data folder

In [6]:

import pandas as pd

# Read the exported JSON file from the data folder
df_coffee = pd.read_json('D:\ironhack\Coursework\week_7\labs\lab-apis\data\coffe_shops.json')
df_coffee.head()



<>:4: SyntaxWarning: invalid escape sequence '\i'
<>:4: SyntaxWarning: invalid escape sequence '\i'
C:\Users\edumc\AppData\Local\Temp\ipykernel_31848\3722138061.py:4: SyntaxWarning: invalid escape sequence '\i'
  df_coffee = pd.read_json('D:\ironhack\Coursework\week_7\labs\lab-apis\data\coffe_shops.json')


,name,coordinates,address,distance,categories,hours,fsq_id
0,El Jardín de Salvador Bachiller,"{'latitude': 40.419109, 'longitude': -3.702326...","Calle Montera, 37, 28013 Madrid Comunidad de M...",287,[Café],NaN,53ce9a52498e0eaa7ccfb45f
1,Chocolatería San Ginés,"{'latitude': 40.416821, 'longitude': -3.706735}","Pasadizo de San Ginés, 5 (C. Arenal), 28013 Ma...",295,"[Café, Coffee Shop, Deli]",NaN,4adcda35f964a5207d3b21e3
2,La Rollerie,"{'latitude': 40.416473, 'longitude': -3.707112}","Calle Mayor, 24 (Coloreros 3), 28013 Madrid Co...",299,[Breakfast Spot],NaN,5564344c498ead26929c9919
3,Valdemeso,"{'latitude': 40.419129, 'longitude': -3.704615}","Mesonero Romanos, 8, 28013 Madrid Comunidad de...",301,"[Restaurant, Café]",NaN,4bb23a3742959c74e08b202c
4,Chocolateria 1902,"{'latitude': 40.417564, 'longitude': -3.706882...","Calle San Martín, 2, 28013 Madrid Comunidad de...",307,"[Coffee Shop, Restaurant]",NaN,514d73fae4b0f2f188ab8d1c


# 2. Plot a map with the coffee shops

In [4]:
from geopy.geocoders import Nominatim
import requests
import folium
import json

def foursquare_places(venue, city, token):
    """
    Search for places of a given type in a specified city using Foursquare API
    Returns a list of tuples containing (name, latitude, longitude)
    """
    # Step 1: Get coordinates from city name
    geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.geocode(city)

    if not location:
        print(f"Could not find location for city: {city}")
        return []

    coordinates = f"{location.latitude},{location.longitude}"
    print(f"City coordinates: {coordinates}")

    # Step 2: Use Foursquare API
    url = f"https://api.foursquare.com/v3/places/search?query={venue}&ll={coordinates}&sort=DISTANCE&limit=10"

    headers = {
        "Accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Foursquare API error: {response.status_code}")
        return []

    data = response.json()
    results = data.get("results", [])
    print(f"\nFound {len(results)} {venue} locations in {city}")
    
    # Create list of places with name and coordinates
    places = []
    for place in results:
        name = place.get("name")
        lat = place.get("geocodes", {}).get("main", {}).get("latitude")
        lng = place.get("geocodes", {}).get("main", {}).get("longitude")
        places.append((name, lat, lng))
    
    # Create and save the map
    create_map(city, location, results, venue)
    
    # Display detailed information
    display_place_details(results)
    
    # Save the full JSON response
    with open("foursquare_response.json", "w") as f:
        json.dump(data, f, indent=2)
    print("\nFull response saved to foursquare_response.json")
    
    return places

def create_map(city, location, results, venue):
    """Create and save an interactive map with markers for each place"""
    map_center = [location.latitude, location.longitude]
    my_map = folium.Map(location=map_center, zoom_start=14)

    # Add a marker for the city center
    folium.Marker(
        location=map_center,
        popup=f"{city} Center",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(my_map)

    # Add markers for each venue
    for place in results:
        name = place.get("name")
        
        # Get location data
        location_data = place.get("location", {})
        address = location_data.get("formatted_address", "Address not available")
        
        # Get coordinates
        lat = place.get("geocodes", {}).get("main", {}).get("latitude")
        lng = place.get("geocodes", {}).get("main", {}).get("longitude")
        
        # Get distance
        distance = place.get("distance", "N/A")
        
        # Get hours
        hours = place.get("hours", {}).get("display", "Hours not available")
        
        # Create popup content
        popup_content = f"""
        <b>{name}</b><br>
        Address: {address}<br>
        Distance: {distance} meters<br>
        Hours: {hours}<br>
        Coordinates: {lat}, {lng}
        """
        
        # Add marker to map
        folium.Marker(
            location=[lat, lng],
            popup=folium.Popup(popup_content, max_width=300),
            icon=folium.Icon(color='green', icon='coffee', prefix='fa')
        ).add_to(my_map)

    # Step 4: Save the map to an HTML file
    map_file = f"{venue.lower()}_{city.lower()}_map.html"
    my_map.save(map_file)
    print(f"\nMap saved to {map_file}")

def display_place_details(results):
    """Display detailed information about each place in the console"""
    for i, place in enumerate(results, 1):
        print(f"\n{i}. {place.get('name')}")
        
        # Location details
        location_data = place.get("location", {})
        print(f"   Address: {location_data.get('formatted_address', 'Not available')}")
        
        # Distance
        distance = place.get("distance")
        if distance:
            print(f"   Distance: {distance} meters")
        
        # Hours
        hours = place.get("hours", {}).get("display")
        if hours:
            print(f"   Hours: {hours}")
        
        # Coordinates
        lat = place.get("geocodes", {}).get("main", {}).get("latitude")
        lng = place.get("geocodes", {}).get("main", {}).get("longitude")
        print(f"   Coordinates: {lat}, {lng}")

# 👇 Interactive usage
if __name__ == "__main__":
    venue_input = input("Enter the type of place you're looking for (e.g., Starbucks, restaurant): ")
    city_input = input("Enter the city: ")
    
    # Get Foursquare API token
    api_token = input("Enter your Foursquare API token: ")
    
    places = foursquare_places(venue_input, city_input, api_token)
    
    if places:
        print("\nTop matching places:")
        for name, lat, lng in places:
            print(f"{name} - Lat: {lat}, Lng: {lng}")
    else:
        print("No places found.")

City coordinates: 40.4167047,-3.7035825

Found 10 coffee locations in Madrid

Map saved to coffee_madrid_map.html

1. Mimi's Creperia
   Address: Calle Postas, 17, 28012 Madrid Comunidad de Madrid
   Distance: 248 meters
   Coordinates: 40.415673, -3.706148

2. El Jardín de Salvador Bachiller
   Address: Calle Montera, 37, 28013 Madrid Comunidad de Madrid
   Distance: 287 meters
   Coordinates: 40.419109, -3.702326

3. Chocolatería San Ginés
   Address: Pasadizo de San Ginés, 5 (C. Arenal), 28013 Madrid Comunidad de Madrid
   Distance: 295 meters
   Coordinates: 40.416821, -3.706735

4. La Rollerie
   Address: Calle Mayor, 24 (Coloreros 3), 28013 Madrid Comunidad de Madrid
   Distance: 299 meters
   Coordinates: 40.416473, -3.707112

5. Valdemeso
   Address: Mesonero Romanos, 8, 28013 Madrid Comunidad de Madrid
   Distance: 301 meters
   Coordinates: 40.419129, -3.704615

6. Chocolateria 1902
   Address: Calle San Martín, 2, 28013 Madrid Comunidad de Madrid
   Distance: 307 meters
   C

After pushing, view your own notebook on [nbviewer](https://nbviewer.org/)